# Fine-Tuning P.1203 Model

# ITU-T P.1203

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from itu_p1203 import P1203Standalone
from itu_p1203 import P1203Pq
from itu_p1203 import P1203Pa
from itu_p1203 import P1203Pv
from itertools import permutations

import json


In [ ]:
f = open("./mode0-bothtests.json")
input_data = json.load(f)
input_data

In [ ]:
# Addendum 07.11: So I believe the bitrate values are all wrong here
# Firstly, Minh also appears to have used mbps in the p1203 calculation, which is wrong (or maybe was right at the time of the previous fine-tuning)
# Now the p1203 model clearly uses kbits/second

# I am collecting all video bitrates in bytes and then converting them to mbits/second, before multiplying them by 1000 to get kbits/second
# I am also only storing the bitrate in kbits to avoid confusion

# The way to get the megabits values is:
# 1. get the size of the bin file in bytes (or kilobytes)
# 2. convert it to megabits by dividing by 125000
# 3. divide it by 10 (since the bin file is for 10-seconds) to get megabits / second
# 4. multiply it by 1000 to get kbit/s
# All of our sequences are 10-second sequences, thus we divide by 10

In [ ]:
import os

bitstream_folder = "./bitstream-files/"
for file_name in os.listdir(bitstream_folder):
    file_path = os.path.join(bitstream_folder, file_name)

    if os.path.isfile(file_path):
        file_size_kb = os.path.getsize(file_path) * 8 / 1024
        print(f"{file_name}: {file_size_kb:.2f} kilobits")
        #object_name = file_name.split(".")[0].split("-")[0]
        #quality = file_name.split(".")[0].split("-")[1]
        #print("\'" + object_name + "\'" + ": " + "{ " + "\'" + quality + "\'" + ": " + f"{file_size_kb:.2f}" + ", ")

In [ ]:
# bitratesMbpsNew = {
#     'BlueSpin': {'r01': 1.75, 'r02': 2.29, 'r03': 3.17, 'r04': 7.53, 'r05': 12.62},
#     'CasualSquat':  {'r01': 2.50, 'r02': 3.91, 'r03': 6.66, 'r04': 19.51, 'r05': 31.11},
#     'ReadyForWinter':  {'r01': 1.97, 'r02': 2.58, 'r03': 3.61, 'r04': 9.34, 'r05': 16.59},
#     'FlowerDance':  {'r01': 2.66, 'r02': 4.24, 'r03': 7.12, 'r04': 22.81, 'r05': 39.35},
# }

bitrates_kbps_new = {
    'BlueSpin': {'r01': 14000.33, 'r02': 18358.77, 'r03': 25381.74, 'r04': 60218.56, 'r05': 100957.13},
    'CasualSquat':  {'r01': 20028.41, 'r02': 31350.91, 'r03': 53280.41, 'r04': 156102.30, 'r05': 248934.89},
    'ReadyForWinter': { 'r01': 15823.20, 'r02': 20687.30, 'r03': 28919.76, 'r04': 74742.73, 'r05': 132748.28 },
    'FlowerDance': { 'r01': 21305.45,  'r02': 33934.77,  'r03': 57028.55, 'r04': 182492.95, 'r05': 314833.07}
}

resolution_map_new = {
    'r01': '640x360', # worse than 420p
    'r02': '852x480', # exactly 480p
    'r03': '1192x672', # bit worse than 720p
    'r04': '1533x864', # a bit better than 720p
    'r05': '1920x1080', # exactly 1080p

}

# bitratesMbpsOld = {
#     'LongDress': {'r1': 4.64, 'r3': 14.05, 'r5':46.78},
#     'Loot': {'r1': 2.28, 'r3': 5.63, 'r5': 16.68},
#     'RedAndBlack': {'r1': 3.39, 'r3': 7.55, 'r5': 22.9},
#     'Soldier': {'r1': 4.38, 'r3': 11.58, 'r5': 35.29}
# }

bitrates_kbps_old = {
    'LongDress': { 'r1': 45330.51, 'r3': 137209.71, 'r5': 456825.40 },
    'Loot': { 'r1': 22223.74, 'r3': 54941.94, 'r5': 162842.81 },
    'RedAndBlack': { 'r1': 33068.13, 'r3': 73753.55, 'r5': 223673.06 },
    'Soldier': { 'r1': 42745.16, 'r3': 113102.48, 'r5': 344626.97 }
}

resolution_map_old = {
    'r1': '854x480', # old dataset 480p
    'r3': '1280x720', # old dataset 720p
    'r5': '1920x1080' # old dataset 1080p
}

In [ ]:
p1203_results = {
    'object': [],
    'start_qual' : [],
    'end_qual' : [],
    'start_bitrate': [],
    'end_bitrate': [],
    'p1203_qoe': []
}

for video in list(bitrates_kbps_old.keys()):
    bitrate_permutations = permutations(list(bitrates_kbps_old[video].values()), 2)
    quality_permutations = permutations(list(bitrates_kbps_old[video].keys()),2)

    bitrates=list(bitrate_permutations)
    qualities = list(quality_permutations)

    for bitrate in list(bitrates_kbps_old[video].values()):
        bitrates.append((bitrate, bitrate))

    for quality in list(bitrates_kbps_old[video].keys()):
        qualities.append((quality, quality))

    for bitrate, quality in zip(bitrates, qualities):
        p1203_results['object'].append(video)
        p1203_results['start_qual'].append(quality[0])
        p1203_results['end_qual'].append(quality[1])
        p1203_results['start_bitrate'].append(bitrate[0])
        p1203_results['end_bitrate'].append(bitrate[1])

        input_data['I13']['segments'][0]['bitrate'] = bitrate[0]
        input_data['I13']['segments'][0]['resolution'] = resolution_map_old[quality[0]]
        input_data['I13']['segments'][1]['bitrate'] = bitrate[1]
        input_data['I13']['segments'][1]['resolution'] = resolution_map_old[quality[1]]

        # the old dataset videos are 30 fps and 10 seconds in sequence length
        input_data['I13']['segments'][0]['fps'] = 30
        input_data['I13']['segments'][1]['fps'] = 30
        input_data['I13']['segments'][0]['duration'] = 5
        input_data['I13']['segments'][1]['duration'] = 5
        input_data['I13']['segments'][1]['start'] = 5

        p1203_results['p1203_qoe'].append(P1203Standalone(input_data).calculate_complete()['O46'])

        print(video)
        print(input_data)
        

for video in list(bitrates_kbps_new.keys()):   
    bitrates = list(bitrates_kbps_new[video].values())
    qualities = list(bitrates_kbps_new[video].keys())

    for bitrate, quality in zip(bitrates, qualities):
        p1203_results['object'].append(video)
        p1203_results['start_qual'].append(quality)
        p1203_results['end_qual'].append(quality)
        p1203_results['start_bitrate'].append(bitrate)
        p1203_results['end_bitrate'].append(bitrate)

        input_data['I13']['segments'][0]['bitrate'] = bitrate
        input_data['I13']['segments'][0]['resolution'] = resolution_map_new[quality]
        input_data['I13']['segments'][1]['bitrate'] = bitrate
        input_data['I13']['segments'][1]['resolution'] = resolution_map_new[quality]

        # the ComPEQ-MR videos are 25 fps and 20 seconds in sequence length
        input_data['I13']['segments'][0]['fps'] = 25
        input_data['I13']['segments'][1]['fps'] = 25
        input_data['I13']['segments'][0]['duration'] = 10
        input_data['I13']['segments'][1]['duration'] = 10
        input_data['I13']['segments'][1]['start'] = 10 # make sure to mark the start of the second segment at 10 seconds for new videos

        p1203_results['p1203_qoe'].append(P1203Standalone(input_data).calculate_complete()['O46'])

        print(video)
        print(input_data)


p1203_results

In [ ]:
p1203_df = pd.DataFrame.from_dict(p1203_results)
p1203_df

In [ ]:
p1203_df_1 = p1203_df.loc[p1203_df['object'].isin(['BlueSpin', 'CasualSquat', 'LongDress', 'Loot'])]
grouped_p1203_qoe = p1203_df_1.groupby(['object', 'start_qual', 'end_qual'],as_index=False)
grouped_p1203_qoe = grouped_p1203_qoe.mean()
grouped_p1203_qoe


In [ ]:
p1203_qoe = list(grouped_p1203_qoe['p1203_qoe'])
p1203_qoe

In [ ]:
import pandas as pd
import numpy as np
# get input data
ground_truth_df = pd.read_csv('./results/test_scores.csv')
ground_truth_df

In [ ]:
def split_column(object_string):
    if 'BlueSpin' in object_string or 'FlowerDance' in object_string or 'ReadyForWinter' in object_string or 'CasualSquat' in object_string:
        split_string = object_string.split('_')
        split_string[1] = split_string[2]
        return split_string
    elif 'LongDress' in object_string or 'Loot' in object_string or 'Soldier' in object_string or 'RedAndBlack' in object_string:
        return object_string.split("_")

ground_truth_df[['object', 'start_qual', 'end_qual', 'dist']] = pd.DataFrame(ground_truth_df['objects'].apply(split_column).to_list(), index=ground_truth_df.index)
ground_truth_df

In [ ]:


#ground_truth_df[['object', 'encoder', 'qual', 'dist']] = ground_truth_df.objects.str.split('_', expand=True)
#df['object'] = df['object'].str.lower()
ground_truth_df['rate_scaled'] = ground_truth_df['rate'] # no need to scale it since this time we took 1-5 results
ground_truth_df

In [ ]:
ground_truth_df = ground_truth_df.loc[ground_truth_df['object'].isin(['BlueSpin', 'CasualSquat', 'LongDress', 'Loot'])]
grouped_ground_truth = ground_truth_df.groupby(['object', 'start_qual', 'end_qual'],as_index=False)
grouped_ground_truth = grouped_ground_truth.mean(numeric_only=True)
grouped_ground_truth

In [ ]:
qoe_ground_truth = list(grouped_ground_truth['rate_scaled'])
qoe_ground_truth

In [ ]:
p1203_qoe

In [ ]:
plt.scatter(qoe_ground_truth, p1203_qoe)
plt.plot([1, 5], [1, 5], color = 'red')
plt.axis([1, 5, 1, 5])
plt.xlabel("Ground Truth")
plt.ylabel("P1203 Mode 0")
plt.savefig("./figures/initial_scatter_plot_BOTH.pdf")

In [ ]:
#import seaborn as sns
df_correlation = pd.DataFrame({'Ground truth': qoe_ground_truth,
                              'P.1203': p1203_qoe})

spear_corr = df_correlation.corr(method = 'spearman') #pearson
print("Spearmann")
print(spear_corr)

#sns.heatmap(corr, cmap='YlGnBu', annot = True)
plt.imshow(spear_corr, cmap='YlGnBu')

plt.text(0, 0, spear_corr['Ground truth']['Ground truth'], ha = "center", va = "center", color = "w")
plt.text(1, 0, spear_corr['Ground truth']['P.1203'], ha = "center", va = "center", color = "black")
plt.text(0, 1, spear_corr['P.1203']['Ground truth'], ha = "center", va = "center", color = "black")
plt.text(1, 1, spear_corr['P.1203']['P.1203'], ha = "center", va = "center", color = "w")

plt.savefig("./figures/correlation_spearman_1203_1080p" + "groundtruth" + "_BOTH.pdf")
plt.show()

pearson_corr = df_correlation.corr(method = 'pearson') #spearman
print("Pearson")
print(pearson_corr)

#sns.heatmap(corr, cmap='YlGnBu', annot = True)
plt.imshow(pearson_corr, cmap='YlGnBu')

plt.text(0, 0, pearson_corr['Ground truth']['Ground truth'], ha = "center", va = "center", color = "w")
plt.text(1, 0, pearson_corr['Ground truth']['P.1203'], ha = "center", va = "center", color = "black")
plt.text(0, 1, pearson_corr['P.1203']['Ground truth'], ha = "center", va = "center", color = "black")
plt.text(1, 1, pearson_corr['P.1203']['P.1203'], ha = "center", va = "center", color = "w")

plt.savefig("./figures/correlation_pearson_1203_1080p" + "groundtruth" + "_BOTH.pdf")
plt.show()



# P1203.1 Pv coefficients updates


In [ ]:
# Quant(a1, a2, a3, a4) = a1 + a2*ln(a3+ln(bitrate) + ln(br*br/(codRes*fr) + a4))

# MOSq(q1, q2, q3) = q1 + q2*exp(q3*Quant)
# MOSq = max(min(MOSq, 5), 1)

# Dq = max(min(100 - RfromMOS(MOSq), 100), 0)

# MOSfromR:
#     MOS = MOS_min + (MOS_max - MOS_min) * R/100 + R*(R-60)(100-R)*0.000007
#     MOS = min(MOS_max, max(MOS, MOS_min))
#     Calculate MOSfromR, R from 0 -> 100, step 0.25

In [ ]:
from functools import lru_cache
from itertools import permutations
import json
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from itu_p1203 import P1203Standalone
from itu_p1203 import P1203Pv


In [ ]:
f = open("./mode0-bothtests.json")
input_data = json.load(f)
# input_data

In [ ]:
ground_truth_qoe_df = pd.read_csv('./results/test_scores.csv')
#ground_truth_qoe_df[['object', 'encoder', 'qual', 'dist']] = ground_truth_qoe_df.objects.str.split('_', expand=True)
ground_truth_qoe_df[['object', 'start_qual', 'end_qual', 'dist']] = pd.DataFrame(ground_truth_qoe_df['objects'].apply(split_column).to_list(), index=ground_truth_qoe_df.index)
ground_truth_qoe_df
ground_truth_qoe_df['rate_scaled'] = ground_truth_qoe_df['rate']

ground_truth_qoe_df

In [ ]:
ground_truth_qoe_df = ground_truth_qoe_df[['object', 'start_qual', 'end_qual', 'rate_scaled']]
ground_truth_qoe_df

In [ ]:
# remove outliers using boxplot method
vpcc_group_columns = ['object', 'start_qual', 'end_qual']
configurations = ground_truth_qoe_df.groupby(vpcc_group_columns,as_index=False)

def boxplot_outlier_filter(frame):
    """
    Outlier filter using interquantile range (filter below Q1 - 1.5 IQR and above Q3 + 1.5 IQR)

    :param frame: data frame
    :return: filtered frame
    """
    q1 = frame.quantile(0.25, numeric_only=True)["rate_scaled"]
    q3 = frame.quantile(0.75, numeric_only=True)["rate_scaled"]
    
    # interquantile range
    iqr = q3 - q1
    fence_low = q1 - (1.5*iqr)
    fence_high = q3 + (1.5*iqr)
    filtered = (frame["rate_scaled"] >= fence_low) & (frame["rate_scaled"] <= fence_high)
    return frame.loc[filtered]

# for each configuration, filter outliers
df_vpcc_filtered = None
for _, frame in configurations:
    #print(boxplot_outlier_filter(frame))
    df_vpcc_filtered = pd.concat([df_vpcc_filtered, boxplot_outlier_filter(frame)], axis=0)

df_vpcc_filtered = df_vpcc_filtered.reset_index(drop=True)
# groups = df_vpcc_filtered.groupby(vpcc_group_columns,as_index=False)


In [ ]:
ground_truth_qoe_df = df_vpcc_filtered.loc[df_vpcc_filtered['object'].isin(['BlueSpin', 'CasualSquat', 'LongDress', 'Loot'])]

# as_index = False to avoid subcolumns
ground_truth_qoe_grouped_df = ground_truth_qoe_df.groupby(['object', 'start_qual', 'end_qual'],as_index=False).aggregate(lambda x: tuple(x))

ground_truth_qoe_grouped_df

In [ ]:
# bitratesMbpsNew = {
#     'BlueSpin': {'r01': 1.75, 'r02': 2.29, 'r03': 3.17, 'r04': 7.53, 'r05': 12.62},
#     'CasualSquat':  {'r01': 2.50, 'r02': 3.91, 'r03': 6.66, 'r04': 19.51, 'r05': 31.11},
#     'ReadyForWinter':  {'r01': 1.97, 'r02': 2.58, 'r03': 3.61, 'r04': 9.34, 'r05': 16.59},
#     'FlowerDance':  {'r01': 2.66, 'r02': 4.24, 'r03': 7.12, 'r04': 22.81, 'r05': 39.35},
# }

bitrates_kbps_new = {
    'BlueSpin': {'r01': 14000.33, 'r02': 18358.77, 'r03': 25381.74, 'r04': 60218.56, 'r05': 100957.13},
    'CasualSquat':  {'r01': 20028.41, 'r02': 31350.91, 'r03': 53280.41, 'r04': 156102.30, 'r05': 248934.89},
    'ReadyForWinter': { 'r01': 15823.20, 'r02': 20687.30, 'r03': 28919.76, 'r04': 74742.73, 'r05': 132748.28 },
    'FlowerDance': { 'r01': 21305.45,  'r02': 33934.77,  'r03': 57028.55, 'r04': 182492.95, 'r05': 314833.07}
}

resolution_map_new = {
    'r01': '640x360', # worse than 420p
    'r02': '852x480', # exactly 480p
    'r03': '1192x672', # bit worse than 720p
    'r04': '1533x864', # a bit better than 720p
    'r05': '1920x1080', # exactly 1080p

}

# bitratesMbpsOld = {
#     'LongDress': {'r1': 4.64, 'r3': 14.05, 'r5':46.78},
#     'Loot': {'r1': 2.28, 'r3': 5.63, 'r5': 16.68},
#     'RedAndBlack': {'r1': 3.39, 'r3': 7.55, 'r5': 22.9},
#     'Soldier': {'r1': 4.38, 'r3': 11.58, 'r5': 35.29}
# }

bitrates_kbps_old = {
    'LongDress': { 'r1': 45330.51, 'r3': 137209.71, 'r5': 456825.40 },
    'Loot': { 'r1': 22223.74, 'r3': 54941.94, 'r5': 162842.81 },
    'RedAndBlack': { 'r1': 33068.13, 'r3': 73753.55, 'r5': 223673.06 },
    'Soldier': { 'r1': 42745.16, 'r3': 113102.48, 'r5': 344626.97 }
}

resolution_map_old = {
    'r1': '854x480', # old dataset 480p
    'r3': '1280x720', # old dataset 720p
    'r5': '1920x1080' # old dataset 1080p
}

In [ ]:
def calculate_p1203(current_coeffs):
    p1203_results = {
        'object': [],
        'start_qual' : [],
        'end_qual' : [],
        'start_bitrate': [],
        'end_bitrate': [],
        'p1203_qoe': []
    }

    # _COEFFS = {
    #     "u1": 72.61,
    #     "u2": 0.32,
    #     "t1": 30.98,
    #     "t2": 1.29,
    #     "t3": 64.65,
    #     "q1": 4.66,
    #     "q2": -0.07,
    #     "q3": 4.06,
    #     "mode0": {
    #         "a1": 11.9983519,
    #         "a2": -2.99991847,
    #         "a3": 41.2475074001,
    #         "a4": 0.13183165961,
    #     },
    #     "mode1": {
    #         "a1": 5.00011566,
    #         "a2": -1.19630824,
    #         "a3": 41.3585049,
    #         "a4": 0,
    #         "c0": -0.91562479,
    #         "c1": 0,
    #         "c2": -3.28579526,
    #         "c3": 20.4098663,
    #     },
    #     "htv_1": -0.60293,
    #     "htv_2": 2.12382,
    #     "htv_3": -0.36936,
    #     "htv_4": 0.03409,
    # }

    # current_coeffs = _COEFFS

    for video in list(bitrates_kbps_old.keys()):
        bitrate_permutations = permutations(list(bitrates_kbps_old[video].values()), 2)
        quality_permutations = permutations(list(bitrates_kbps_old[video].keys()),2)

        bitrates=list(bitrate_permutations)
        qualities = list(quality_permutations)

        for bitrate in list(bitrates_kbps_old[video].values()):
            bitrates.append((bitrate, bitrate))

        for quality in list(bitrates_kbps_old[video].keys()):
            qualities.append((quality, quality))

        for bitrate, quality in zip(bitrates, qualities):
            p1203_results['object'].append(video)
            p1203_results['start_qual'].append(quality[0])
            p1203_results['end_qual'].append(quality[1])
            p1203_results['start_bitrate'].append(bitrate[0])
            p1203_results['end_bitrate'].append(bitrate[1])

            input_data['I13']['segments'][0]['bitrate'] = bitrate[0]
            input_data['I13']['segments'][0]['resolution'] = resolution_map_old[quality[0]]
            input_data['I13']['segments'][1]['bitrate'] = bitrate[1]
            input_data['I13']['segments'][1]['resolution'] = resolution_map_old[quality[1]]

            # the old dataset videos are 30 fps and 10 seconds in sequence length
            input_data['I13']['segments'][0]['fps'] = 30
            input_data['I13']['segments'][1]['fps'] = 30
            input_data['I13']['segments'][0]['duration'] = 5
            input_data['I13']['segments'][1]['duration'] = 5
            input_data['I13']['segments'][1]['start'] = 5

            p1203_qoe = P1203Standalone(input_data, coeffs=current_coeffs).calculate_complete()['O46']

            p1203_results['p1203_qoe'].append(p1203_qoe)

            #print(video)
            print("input_data")
            print(input_data)
            print("p1203_qoe: " + str(p1203_qoe))

    for video in list(bitrates_kbps_new.keys()):   
        bitrates = list(bitrates_kbps_new[video].values())
        qualities = list(bitrates_kbps_new[video].keys())

        for bitrate, quality in zip(bitrates, qualities):
            p1203_results['object'].append(video)
            p1203_results['start_qual'].append(quality)
            p1203_results['end_qual'].append(quality)
            p1203_results['start_bitrate'].append(bitrate)
            p1203_results['end_bitrate'].append(bitrate)

            input_data['I13']['segments'][0]['bitrate'] = bitrate
            input_data['I13']['segments'][0]['resolution'] = resolution_map_new[quality]
            input_data['I13']['segments'][1]['bitrate'] = bitrate
            input_data['I13']['segments'][1]['resolution'] = resolution_map_new[quality]

            # the ComPEQ-MR videos are 25 fps and 20 seconds in sequence length
            input_data['I13']['segments'][0]['fps'] = 25
            input_data['I13']['segments'][1]['fps'] = 25
            input_data['I13']['segments'][0]['duration'] = 10
            input_data['I13']['segments'][1]['duration'] = 10
            input_data['I13']['segments'][1]['start'] = 10 # make sure to mark the start of the second segment at 10 seconds for new videos

            p1203_qoe = P1203Standalone(input_data, coeffs=current_coeffs).calculate_complete()['O46']

            p1203_results['p1203_qoe'].append(p1203_qoe)

            #print(video)
            print("input_data")
            print(input_data)
            print("p1203_qoe: " + str(p1203_qoe))

    #print("p1203_results: " + p1203_results)
    return p1203_results

In [ ]:
def calculate_rmse(p1203_results, mos):
    #print("p1203_results")
    #print(p1203_results)
    #print("mos")
    #print(mos)
    joined_qoe = p1203_results.merge(mos)
    rmse_arr = []
    #print("joined_qoe")
    #print(joined_qoe)

    for idx in np.arange(joined_qoe.shape[0]):
        
        #print("joined_qoe.shape[0]")
        #print(joined_qoe.shape[0])
        targets = joined_qoe.iloc[idx,:]['rate_scaled']
        #print("targets")
        #print(targets)
        #print(len(targets))
        predictions = np.full(len(targets), joined_qoe.loc[joined_qoe.index[idx], 'p1203_qoe'])
        #print("predictions")
        #print(predictions)
        #print(len(predictions))
        rmse_arr.append(np.sqrt(np.mean((predictions-targets)**2)))
        #print("rmse_arr")
        #print(rmse_arr)
    
    return np.average(rmse_arr)


In [ ]:
from datetime import datetime

_COEFFS = {
        "u1": 72.61,
        "u2": 0.32,
        "t1": 30.98,
        "t2": 1.29,
        "t3": 64.65,
        "q1": 4.66,
        "q2": -0.07,
        "q3": 4.06,
        "mode0": {
            "a1": 11.9983519,
            "a2": -2.99991847,
            "a3": 41.2475074001,
            "a4": 0.13183165961,
        },
        "mode1": {
            "a1": 5.00011566,
            "a2": -1.19630824,
            "a3": 41.3585049,
            "a4": 0,
            "c0": -0.91562479,
            "c1": 0,
            "c2": -3.28579526,
            "c3": 20.4098663,
        },
        "htv_1": -0.60293,
        "htv_2": 2.12382,
        "htv_3": -0.36936,
        "htv_4": 0.03409,
    }

# # this function isnt being called normally anywhere
# def finetune_coeffs():
#     min_rmse = sys.float_info.max
#     optimized_coeffs = _COEFFS
#     optimized_p1203_qoe  =[]

#     # for _COEFFS['u1'] in np.arange(50,100):
#     #     for _COEFFS['u2'] in np.arange(1):
#             # for _COEFFS['t1'] in np.arange(1):
#             #     for _COEFFS['t2'] in np.arange(1):
#             #         for _COEFFS['t3'] in np.arange(1):
#                         # for _COEFFS['q1'] in np.arange(1):
#                         #     for _COEFFS['q2'] in np.arange(1):
#                         #         for _COEFFS['q3'] in np.arange(1):
#     for _COEFFS['mode0']['a1'] in np.arange(1, 15, 0.5): # 0.5 range originally
#         for _COEFFS['mode0']['a2'] in np.arange(-5, 0, 0.5): # 0.5 range originally
#             for _COEFFS['mode0']['a3'] in np.arange(0, 80, 1): # 1 range originally
#                 for _COEFFS['mode0']['a4'] in np.arange(0, 5, 0.5): # 0.5 range originally
#                     # print(_COEFFS)

#                     qoe_p1203_dict = calculate_p1203(_COEFFS)
#                     p1203_df = pd.DataFrame.from_dict(qoe_p1203_dict)

#                     # Fine tune aka train with BlueSpin and CasualSquat
#                     p1203_df = p1203_df.loc[p1203_df['object'].isin(['BlueSpin', 'CasualSquat', 'LongDress', 'Loot'])]
#                     grouped_p1203_qoe = p1203_df.groupby(['object', 'start_qual', 'end_qual'],as_index=False).mean()
#                     #print("grouped_p1203_qoe")
#                     #print(grouped_p1203_qoe)

#                     print("timestamp")
#                     print(datetime.now())
#                     print("COEFFS: " + str(_COEFFS['mode0']['a1']) + " " + str(_COEFFS['mode0']['a2']) + " " + str(_COEFFS['mode0']['a3']) + " " + str(_COEFFS['mode0']['a4']))

#                     rmse = calculate_rmse(grouped_p1203_qoe, ground_truth_qoe_grouped_df)
#                     print("Calcd RMSE: " + str(rmse))
#                     print("Current min RMSE: " + str(min_rmse))
#                     if rmse < min_rmse:   
#                         print("==> NEW MIN RMSE: " + str(rmse))
#                         print("==> MIN RMSE _COEFFS: " + str(_COEFFS))
                        
#                         optimized_coeffs = _COEFFS
#                         min_rmse = rmse
#                         optimized_p1203_qoe = list(grouped_p1203_qoe['p1203_qoe'])

#     return optimized_coeffs, optimized_p1203_qoe, min_rmse

In [29]:
from scipy.optimize import minimize, Bounds, differential_evolution, basinhopping, brute, shgo, dual_annealing, direct

# objective function to minimise
def objective_function(trial_coeffs, multiplier=1):
    print('NEW OBJECTIVE FUNCTION CALL')

    current_coeffs = _COEFFS
    current_coeffs['u1'] = trial_coeffs[0].item()
    current_coeffs['u2'] = trial_coeffs[1].item()
    current_coeffs['t1'] = trial_coeffs[2].item()
    current_coeffs['t2'] = trial_coeffs[3].item()
    current_coeffs['t3'] = trial_coeffs[4].item()
    current_coeffs['q1'] = trial_coeffs[5].item()
    current_coeffs['q2'] = trial_coeffs[6].item()
    current_coeffs['q3'] = trial_coeffs[7].item()
    current_coeffs['mode0']['a1'] = trial_coeffs[8].item()
    current_coeffs['mode0']['a2'] = trial_coeffs[9].item()
    current_coeffs['mode0']['a3'] = trial_coeffs[10].item()
    current_coeffs['mode0']['a4'] = trial_coeffs[11].item()

    qoe_p1203_dict = calculate_p1203(current_coeffs)
    p1203_df = pd.DataFrame.from_dict(qoe_p1203_dict)
    p1203_df = p1203_df.loc[p1203_df['object'].isin(['BlueSpin', 'CasualSquat', 'LongDress', 'Loot'])]
    grouped_p1203_qoe = p1203_df.groupby(['object', 'start_qual', 'end_qual'],as_index=False).mean()
    print("COEFFS: " + str(current_coeffs['mode0']['a1']) + " " + str(current_coeffs['mode0']['a2']) + " " + str(current_coeffs['mode0']['a3']) + " " + str(current_coeffs['mode0']['a4']))

    rmse = calculate_rmse(grouped_p1203_qoe, ground_truth_qoe_grouped_df)
    print("Calcd RMSE: " + str(rmse))
    #print("Current min RMSE: " + str(min_rmse))
    return rmse

initial_coeffs = [12,-3,41,0]

optimization_methods = ["basinhopping", "brute", "differential_evolution", "shgo", "dual_annealing", "direct"]
optimization_method = "dual_annealing"

bounds = [(50,90), (-0,10), (20,40), (0,10), (50,70), (0,10), (-10,10), (0,10), (1,15), (-5, 5), (0, 80), (0, 5)]
#bounds = [(0,100), (0,100), (0,100), (0,100)]
#bounds=[(-5,80), (-5,80), (-5,80), (-5,80)]
#bounds = [(-5,80), (-5,80), (-5,80), (-5,80),(-5,80), (-5,80), (-5,80), (-5,80),(-5,80), (-5,80), (-5,80), (-5,80)]
#bounds = Bounds(-100,100)

#result = minimize(objective_function, initial_coeffs, method='L-BFGS-B', bounds=bounds)

match optimization_method:
    case "basinhopping":
        result = basinhopping(objective_function, initial_coeffs, minimizer_kwargs={'bounds':bounds})
    case "brute":
        (x0, fval, grid, Jout) = brute(objective_function, bounds, full_output=True)
    case "differential_evolution":
        result = differential_evolution(objective_function, bounds)
    case "shgo":
        result = shgo(objective_function, bounds, sampling_method='halton')
    case "dual_annealing":
        result = dual_annealing(objective_function, bounds)
    case "direct":
        result = basinhopping(objective_function, bounds)    

input_data
{'I11': {'segments': [], 'streamId': 42}, 'I13': {'segments': [{'codec': 'h264', 'start': 0, 'duration': 5, 'resolution': '1280x720', 'bitrate': 7.375355, 'fps': 30}, {'codec': 'h264', 'start': 5, 'duration': 5, 'resolution': '1280x720', 'bitrate': 7.375355, 'fps': 30}], 'streamId': 42}, 'I23': {'stalling': [], 'streamId': 42}, 'IGen': {'device': 'pc', 'displaySize': '1920x1080', 'viewingDistance': '150cm'}}
p1203_qoe: 2.021464810571226
p1203 coeffs {'u1': 57.604041911661625, 'u2': 0.7348794117569923, 't1': 30.67816524580121, 't2': 1.3893132582306862, 't3': 54.60779080539942, 'q1': 1.5986969359219074, 'q2': 3.7051077261567116, 'q3': 4.500655047595501, 'mode0': {'a1': 3.728855572640896, 'a2': -2.984356116503477, 'a3': 22.613381884992123, 'a4': 0.13581175357103348}, 'mode1': {'a1': 5.00011566, 'a2': -1.19630824, 'a3': 41.3585049, 'a4': 0, 'c0': -0.91562479, 'c1': 0, 'c2': -3.28579526, 'c3': 20.4098663}, 'htv_1': -0.60293, 'htv_2': 2.12382, 'htv_3': -0.36936, 'htv_4': 0.03409}


KeyboardInterrupt: 

In [ ]:
print(optimization_method)
if optimization_method == "brute":
    print("x0: " + x0)
    print("fval: " + fval)
    print("grid: " + grid)
    print("Jout: " + Jout)
else: 
    print("optimal coeffs " + optimization_method + ": " + str(result.x))
    print("objective function value " + optimization_method + ": " + str(result.fun))
    print("objective function message " + optimization_method + ": " + str(result.message))

In [ ]:
#optimized_coeffs, p1203_qoe_list, min_rmse =  finetune_coeffs()
#print("timestamp")
#print(datetime.now())
#print("FINAL RESULT")
#print(optimized_coeffs, p1203_qoe_list, min_rmse)

In [ ]:
#print(min_rmse)
#print(optimized_coeffs)
#print(ground_truth_qoe_df.groupby(['object', 'qual'],as_index=False).mean()['rate_scaled'])


In [ ]:
#==============================================================================================================================
#FOUND THE BEST

In [ ]:
ground_truth_qoe_list = list(ground_truth_qoe_df.groupby(['object', 'start_qual', 'end_qual'],as_index=False).mean()['rate_scaled'])
ground_truth_qoe_list = ground_truth_qoe_list

In [ ]:
#coeffs = {'u1': 72.61, 'u2': 0.32, 't1': 30.98, 't2': 1.29, 't3': 64.65, 'q1': 4.4, 'q2': -1.5, 'q3': 1.5, 'mode0': {'a1': 5, 'a2': -1.0, 'a3': 130, 'a4': 4.0}, 'mode1': {'a1': 5.00011566, 'a2': -1.19630824, 'a3': 41.3585049, 'a4': 0, 'c0': -0.91562479, 'c1': 0, 'c2': -3.28579526, 'c3': 20.4098663}, 'htv_1': -0.60293, 'htv_2': 2.12382, 'htv_3': -0.36936, 'htv_4': 0.03409}
coeffs = _COEFFS

results_file = open(optimization_method + ".txt", "a")
results_file.write(optimization_method + " results ")

print(optimization_method + " results ")

results_file.write(str(result.x[0]))
results_file.write(str(result.x[1]))
results_file.write(str(result.x[2]))
results_file.write(str(result.x[3]))

print(result.x[0])
print(result.x[1])
print(result.x[2])
print(result.x[3])

results_file.write(str(result.x))
results_file.write(str(result.fun))


coeffs['mode0']['a1'] = result.x[0].item()
coeffs['mode0']['a2'] = result.x[1].item()
coeffs['mode0']['a3'] = result.x[2].item()
coeffs['mode0']['a4'] = result.x[3].item()
   

#coeffs = optimized_coeffs
print(coeffs)

qoe_p1203_dict = calculate_p1203(coeffs)
p1203_df = pd.DataFrame.from_dict(qoe_p1203_dict)

p1203_df = p1203_df.loc[p1203_df['object'].isin(['BlueSpin', 'CasualSquat', 'LongDress', 'Loot'])]
grouped_p1203_qoe = p1203_df.groupby(['object', 'start_qual', 'end_qual'],as_index=False).mean()
grouped_p1203_qoe

In [ ]:
p1203_qoe_list = list(grouped_p1203_qoe['p1203_qoe'])
plt.scatter(ground_truth_qoe_list, p1203_qoe_list)
plt.plot([1, 5], [1, 5], color = 'red')
plt.axis([1, 5, 1, 5])
plt.xlabel("Ground Truth")
plt.ylabel("Fine-tuned P1203 Mode 0")

In [ ]:
#import seaborn as sns

df_correlation = pd.DataFrame({
    "Ground truth": ground_truth_qoe_list,
    "P.1203": p1203_qoe_list
})

#corr_method = 'spearman'
#corr = df_correlation.corr(method=corr_method)
#sns.heatmap(corr, cmap='YlGnBu', annot = True)
#plt.savefig('./figures/correlation_heatmap_' + corr_method + '_optimized_BOTH.pdf')
#plt.show()

spear_corr = df_correlation.corr(method = 'spearman') #pearson
print("Spearmann")
print(spear_corr)
print(spear_corr.shape)
print(spear_corr["Ground truth"]["P.1203"])

#sns.heatmap(corr, cmap='YlGnBu', annot = True)
plt.imshow(spear_corr, cmap='YlGnBu')

plt.text(0, 0, spear_corr['Ground truth']['Ground truth'], ha = "center", va = "center", color = "w")
plt.text(1, 0, spear_corr['Ground truth']['P.1203'], ha = "center", va = "center", color = "black")
plt.text(0, 1, spear_corr['P.1203']['Ground truth'], ha = "center", va = "center", color = "black")
plt.text(1, 1, spear_corr['P.1203']['P.1203'], ha = "center", va = "center", color = "w")


plt.savefig("./figures/correlation_spearman_optimized_" + optimization_method + "_BOTH.pdf")
plt.show()

pearson_corr = df_correlation.corr(method = 'pearson') #spearman
print("Pearson")
print(pearson_corr)

#sns.heatmap(corr, cmap='YlGnBu', annot = True)
plt.imshow(pearson_corr, cmap='YlGnBu')

plt.text(0, 0, pearson_corr['Ground truth']['Ground truth'], ha = "center", va = "center", color = "w")
plt.text(1, 0, pearson_corr['Ground truth']['P.1203'], ha = "center", va = "center", color = "black")
plt.text(0, 1, pearson_corr['P.1203']['Ground truth'], ha = "center", va = "center", color = "black")
plt.text(1, 1, pearson_corr['P.1203']['P.1203'], ha = "center", va = "center", color = "w")


plt.savefig("./figures/correlation_pearson_optimized_" + optimization_method + "_BOTH.pdf")
plt.show()


In [ ]:
# corr_method = 'pearson'
# corr = df_correlation.corr(method=corr_method)
# sns.heatmap(corr, cmap='YlGnBu', annot = True)
# plt.savefig('./figures/correlation_heatmap_' + corr_method + '_optimized_BOTH.pdf')
# plt.show()

In [ ]:
#==============================================================================================================================
#VALIDATING with ReadyForWinter and FlowerDance

In [ ]:
coeffs = {'u1': 72.61, 'u2': 0.32, 't1': 30.98, 't2': 1.29, 't3': 64.65, 'q1': 4.4, 'q2': -1.5, 'q3': 1.5, 'mode0': {'a1': 5, 'a2': -1.0, 'a3': 130, 'a4': 4.0}, 'mode1': {'a1': 5.00011566, 'a2': -1.19630824, 'a3': 41.3585049, 'a4': 0, 'c0': -0.91562479, 'c1': 0, 'c2': -3.28579526, 'c3': 20.4098663}, 'htv_1': -0.60293, 'htv_2': 2.12382, 'htv_3': -0.36936, 'htv_4': 0.03409}

qoe_p1203_dict = calculate_p1203(coeffs)
qoe_p1203_df = pd.DataFrame(qoe_p1203_dict)
qoe_p1203_df = qoe_p1203_df.loc[qoe_p1203_df['object'].isin(['ReadyForWinter', 'FlowerDance', 'RedAndBlack', 'Soldier'])]
                                # & ((qoe_p1203_df['qual'].isin(['r01']) & qoe_p1203_df['end_quality'].isin(['r1'])) |
                                #  (qoe_p1203_df['start_quality'].isin(['r3']) & qoe_p1203_df['end_quality'].isin(['r3'])) |
                                #  (qoe_p1203_df['start_quality'].isin(['r5']) & qoe_p1203_df['end_quality'].isin(['r5']))) ]
qoe_validation = qoe_p1203_df.groupby(['object', 'start_qual', 'end_qual'],as_index=False).mean()
qoe_validation

In [ ]:
# Get ground truth qoe for these sequences
ground_truth_qoe_df = df_vpcc_filtered.loc[df_vpcc_filtered['object'].isin(['ReadyForWinter', 'FlowerDance', 'RedAndBlack', 'Soldier'])]
ground_truth_qoe_grouped_df = ground_truth_qoe_df.groupby(['object', 'start_qual', 'end_qual'],as_index=False).aggregate(lambda x: tuple(x))
ground_truth_qoe_grouped_df

In [ ]:
calculate_rmse(qoe_validation, ground_truth_qoe_grouped_df)

In [ ]:
ground_truth_list = list(ground_truth_qoe_df.groupby(['object', 'start_qual', 'end_qual'],as_index=False).mean()['rate_scaled'])
qoe_validation_list = list(qoe_validation['p1203_qoe'])
ground_truth_list

In [ ]:
plt.scatter(ground_truth_list, qoe_validation_list)
plt.plot([1, 5], [1, 5], color='red')
plt.axis([1, 5, 1, 5])
plt.xlabel('Ground Truth')
plt.ylabel('Fine-tuned P1203 Mode 0 - Validation')
plt.show()

In [ ]:
corr_method = 'spearman'
corr_df = pd.DataFrame({
    'Fine-tuned P1203': qoe_validation_list,
    'Ground truth': ground_truth_list
})

corr_methods = ['spearman', 'pearson']

for corr_method in corr_methods:
    corr = corr_df.corr(method=corr_method)
    #sns.heatmap(corr, cmap='YlGnBu', annot = True)
    plt.imshow(corr, cmap='YlGnBu')

    plt.text(0, 0, corr['Ground truth']['Ground truth'], ha = "center", va = "center", color = "w")
    plt.text(1, 0, corr['Ground truth']['Fine-tuned P1203'], ha = "center", va = "center", color = "black")
    plt.text(0, 1, corr['Fine-tuned P1203']['Ground truth'], ha = "center", va = "center", color = "black")
    plt.text(1, 1, corr['Fine-tuned P1203']['Fine-tuned P1203'], ha = "center", va = "center", color = "w")

    plt.savefig('./figures/correlation_heatmap_' + corr_method + "_" + optimization_method + '_validation_BOTH.pdf')
    plt.title(corr_method)
    plt.show()

In [ ]:
# original P1203
_COEFFS = {
        "u1": 72.61,
        "u2": 0.32,
        "t1": 30.98,
        "t2": 1.29,
        "t3": 64.65,
        "q1": 4.66,
        "q2": -0.07,
        "q3": 4.06,
        "mode0": {
            "a1": 11.9983519,
            "a2": -2.99991847,
            "a3": 41.2475074001,
            "a4": 0.13183165961,
        },
        "mode1": {
            "a1": 5.00011566,
            "a2": -1.19630824,
            "a3": 41.3585049,
            "a4": 0,
            "c0": -0.91562479,
            "c1": 0,
            "c2": -3.28579526,
            "c3": 20.4098663,
        },
        "htv_1": -0.60293,
        "htv_2": 2.12382,
        "htv_3": -0.36936,
        "htv_4": 0.03409,
    }

qoe_p1203_dict = calculate_p1203(_COEFFS)
qoe_p1203_df = pd.DataFrame(qoe_p1203_dict)
qoe_p1203_df = qoe_p1203_df.loc[qoe_p1203_df['object'].isin(['ReadyForWinter', 'FlowerDance', 'RedAndBlack', 'Soldier'])]
                                #  & 
                                # ((qoe_p1203_df['start_quality'].isin(['r1']) & qoe_p1203_df['end_quality'].isin(['r1'])) |
                                #  (qoe_p1203_df['start_quality'].isin(['r3']) & qoe_p1203_df['end_quality'].isin(['r3'])) |
                                #  (qoe_p1203_df['start_quality'].isin(['r5']) & qoe_p1203_df['end_quality'].isin(['r5']))) ]
qoe_original_p1203 = qoe_p1203_df.groupby(['object', 'start_qual', 'end_qual'],as_index=False).mean()
qoe_original_p1203

In [ ]:
print(calculate_rmse(qoe_original_p1203, ground_truth_qoe_grouped_df))
qoe_original_p1203_list = list(qoe_original_p1203['p1203_qoe'])

In [ ]:
plt.scatter(ground_truth_list, qoe_original_p1203_list)
plt.plot([1, 5], [1, 5], color='red')
plt.axis([1, 5, 1, 5])
plt.xlabel('Ground Truth')
plt.ylabel('Original P1203 Mode 0')
plt.show()

In [ ]:
corr_all_df = pd.DataFrame({
    "Ground truth": ground_truth_list,
    # "Fine-tuned P1203": qoe_validation_list,
    "Original P1203": qoe_original_p1203_list
})

corr_methods = ['spearman', 'pearson']
for corr_method in corr_methods:
    corr = corr_all_df.corr(method=corr_method)
    #sns.heatmap(corr, cmap='YlGnBu', annot=True)
    plt.imshow(corr, cmap='YlGnBu')

    plt.text(0, 0, corr['Ground truth']['Ground truth'], ha = "center", va = "center", color = "w")
    plt.text(1, 0, corr['Ground truth']['Original P1203'], ha = "center", va = "center", color = "black")
    plt.text(0, 1, corr['Original P1203']['Ground truth'], ha = "center", va = "center", color = "black")
    plt.text(1, 1, corr['Original P1203']['Original P1203'], ha = "center", va = "center", color = "w")
    
    plt.title(corr_method)
    plt.show()